In [1]:
import pickle

with open('/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    df = pickle.load(file)

df['protein_id'] = df['pdb_id'] + df['struct_asym_id']
df

,ACC,SEG,ID,up_aas,n_aas,SEG_ID,SITE_NAME,pdb_id,auth_asym_id,struct_asym_id,rep_chain,RoG,aas,centre,SASA,VOL,protein_id
0,P03372,2,0,"[339, 341, 342, 343, 344, 345, 346, 347, 349, ...",61,P03372_2,P03372_2_0,1a52,A,A,1a52_A,14.20,"[43, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 5...","(-0.966, -16.192, -2.54)",3266.99,5651.0,1a52A
1,P03372,2,1,"[329, 330, 331, 332, 333, 335, 336, 337, 338, ...",18,P03372_2,P03372_2_1,1a52,A,A,1a52_A,7.67,"[33, 34, 35, 36, 37, 39, 40, 41, 42, 45, 46, 4...","(7.883, -18.704, 5.375)",1034.84,451.0,1a52A
2,P03372,2,2,"[326, 327, 328, 330, 352]",5,P03372_2,P03372_2_2,1a52,A,A,1a52_A,5.46,"[30, 31, 32, 34, 56]","(2.537, -9.125, 11.7)",460.17,30.0,1a52A
3,P00750,3,0,"[342, 357, 358, 401, 402, 403, 482, 484, 485, ...",27,P00750_3,P00750_3_0,1a5h,B,D,1a5h_B,8.98,"[32, 47, 48, 91, 92, 93, 172, 174, 175, 197, 1...","(0.467, 7.085, -7.455)",968.54,951.0,1a5hD
4,P02100,1,0,"[32, 39, 42, 43, 46, 60, 64, 67, 68, 71, 72, 8...",22,P02100_1,P02100_1_0,1a9w,E,B,1a9w_E,9.81,"[31, 38, 41, 42, 45, 59, 63, 66, 67, 70, 71, 8...","(-4.39, 6.075, 1.831)",895.32,913.0,1a9wB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,P40818,3,0,"[936, 938, 939, 940, 985, 987, 988, 992]",8,P40818_3,P40818_3_0,8xpn,A,A,8xpn_A,5.07,"[203, 205, 206, 207, 252, 254, 255, 259]","(2.034, -33.131, -8.816)",478.35,25.0,8xpnA
8240,P35222,1,0,"[205, 206, 207, 208, 211, 242, 243, 245, 246, ...",12,P35222_1,P35222_1_0,8y14,A,A,8y14_A,6.96,"[68, 69, 70, 71, 74, 105, 106, 108, 109, 110, ...","(1.987, -37.811, -11.715)",762.12,202.0,8y14A
8241,O95970,2,0,"[334, 336, 381, 382, 383]",5,O95970_2,O95970_2_0,8y6b,F,E,8y6b_F,4.21,"[303, 305, 350, 351, 352]","(1.688, 25.917, -1.282)",71.26,25.0,8y6bE
8242,O95970,2,1,"[422, 423, 424, 425]",4,O95970_2,O95970_2_1,8y6b,F,E,8y6b_F,3.77,"[391, 392, 393, 394]","(16.013, 33.482, 2.164)",296.42,0.0,8y6bE


seems like ligysis uses label_seq labeling, not auth! Same for P2Rank predictions!

# loop over PDB structures
go over PDB and read input indexes.

In [ ]:
import sys
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import cryptoshow_utils

binding_sites = {}
for protein_id, binding_site in zip(df['protein_id'], df['aas']):
    if protein_id not in binding_sites:
        binding_sites[protein_id] = []
    binding_sites[protein_id].append(binding_site)

# Translate from author numbering to mmCIF numbering (zero-based)
VERBOSE = False
translated_binding_sites = {}
for protein_id in binding_sites:
    pdb_id, chain_id = protein_id[:4], protein_id[4:]
    translated_binding_sites[protein_id] = []
    for binding_site in binding_sites[protein_id]:
        translated_binding_site = cryptoshow_utils.map_auth_to_mmcif_numbering(pdb_id, chain_id, binding_site, auth=False)
        if VERBOSE:
            # ONLY 35 CASES WITH LENGTH MISMATCH! (out of 8244)
            if len(translated_binding_site[0]) != len(binding_site):
                print(f"Warning: Length mismatch for {protein_id}. Original: {len(binding_site)}, Translated: {len(translated_binding_site[0])}")
            else:
                print(f"Successfully translated binding site for {protein_id}. Length: {len(binding_site)}")
        translated_binding_sites[protein_id].append(translated_binding_site)

# save
save for both the merged pockets for residue-level evaluation, and non-merged pockets for pocket-level evaluation. 

In [4]:
with open("/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_for_pocket_level_evaluation.csv", "w") as file:
    for protein_id in translated_binding_sites:
        for binding_site, sequence in translated_binding_sites[protein_id]:
            file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_site))};{sequence}\n')

In [ ]:
with open("/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_without_unobserved.csv", "w") as file:
    for protein_id in translated_binding_sites:
        binding_residues = set()
        for binding_site, sequence in translated_binding_sites[protein_id]:
            binding_residues.update(binding_site)
        file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_residues))};{sequence}\n')

In [ ]:
import numpy as np
COORDINATES_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'
for protein_id in translated_binding_sites:
    pdb_id, chain_id = protein_id[:4], protein_id[4:]
    coordinates = cryptoshow_utils.get_coordinates(pdb_id, chain_id, auth=False)
    np.save(f'{COORDINATES_PATH}/{protein_id}.npy', coordinates)

In [ ]:
df['pdb_id'].nunique()

3376